In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed May  3 11:18:16 2017

@author: zhouyi
"""

import glob
import os
import sys

import implicit
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse

import evaluation

DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/MSD'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid_sub.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid_sub.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
def load_data(csv_file, shape=(n_songs, n_users)):
    st = pd.read_csv(csv_file)
    cols, rows = np.array(st['uid'], dtype=np.int32), np.array(st['sid'], dtype=np.int32)
    count = st['count']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.sub.csv'))
print 'train_data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.sub.csv'))
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.sub.csv'))
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape

print 'sparse according to vad_data:',float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])

train_data.shape: (400, 1013)
train_data entries: (24587,)
vad__data.shape: (400, 1013)
vad_data entries: (2731,)
test_data.shape: (400, 1013)
test_data entries: (6829,)
sparse according to vad_data: 0.00673988153998


In [4]:
def myEvaluation(model,X,vad_data):

    vad_recall_at_k = evaluation.Recall_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_map_at_k = evaluation.MAP_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_ndcg_at_k = evaluation.NDCG_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
    return vad_ndcg_at_k,vad_recall_at_k,vad_map_at_k
    

In [5]:
# initialize a model
print "---RAW---"
model = implicit.als.AlternatingLeastSquares(factors=25)
model.fit(train_data.astype(np.double))

VAD_RECORD=myEvaluation(model,train_data,vad_data)
print "Validation NDGC@K",VAD_RECORD[0]
print "Validation Recall@K",VAD_RECORD[1]
print "Validation MAP@K",VAD_RECORD[2]

    
TEST_RECORD=myEvaluation(model,train_data,test_data)
print "Test NDGC@K",TEST_RECORD[0]
print "Test Recall@K",TEST_RECORD[1]
print "Test MAP@K",TEST_RECORD[2]

---RAW---
Validation NDGC@K 0.0225686670772
Validation Recall@K 0.194703914052
Validation MAP@K 0.0200506254484
Test NDGC@K 0.0603278446033
Test Recall@K 0.233041945486
Test MAP@K 0.0526991111156
